In [46]:
!nvidia-smi

Mon Jun 13 13:19:15 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.57.02    Driver Version: 470.57.02    CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  On   | 00000000:00:05.0 Off |                  Off |
| N/A   37C    P0    62W / 300W |  16425MiB / 32510MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  Tesla V100-SXM2...  On   | 00000000:00:06.0 Off |                  Off |
| N/A   

In [44]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   
os.environ["CUDA_VISIBLE_DEVICES"]="1"

In [41]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print('Device:', device)
print('Current cuda device:', torch.cuda.current_device())

Device: cuda
Current cuda device: 0


패키지 다운

In [2]:
!pip install transformers datasets
!pip install transformers

In [3]:
import os
from transformers import AutoTokenizer
from tqdm import tqdm
import pandas as pd
from sklearn.model_selection import train_test_split, StratifiedKFold

from itertools import combinations
import random
import numpy as np
import torch

from transformers import AutoTokenizer, Trainer, TrainingArguments, DataCollatorWithPadding, AutoModelForSequenceClassification, EarlyStoppingCallback
from datasets import load_dataset, load_metric
from sklearn.model_selection import train_test_split

In [4]:
def main(seed = 42):
    os.environ['PYTHONHASHSEED'] = str(seed)
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.benchmark = True
    
main()

In [5]:
code = '/home/ubuntu/HwangJaewon/DL_class/code/'
code_file = os.listdir(code)

In [6]:
train = pd.read_csv("/home/ubuntu/HwangJaewon/DL_class/train_data.csv")
train.head()
len(train)

5133767

In [7]:
test = pd.read_csv("/home/ubuntu/HwangJaewon/DL_class/test.csv")
test.head()
len(test)

179700

In [8]:
def preprocess_script(script):
    '''
    간단한 전처리 함수
    주석 -> 삭제
    '    '-> tab 변환
    다중 개행 -> 한 번으로 변환
    '''
    with open(script,'r',encoding='utf-8') as file:
        lines = file.readlines()
        preproc_lines = []
        for line in lines:
            if line.lstrip().startswith('#'): # #으로 시작하면 -> pass
                continue
            line = line.rstrip()
            if '#' in line: # 중간에 #이 있으면 #전까지 indexing
                line = line[:line.index('#')]
            line = line.replace('\n','') # \n -> ''
            line = line.replace('    ','\t') # '     ' -> \t로
            if line == '':
                continue
            preproc_lines.append(line)
        preprocessed_script = '\n'.join(preproc_lines)
    return preprocessed_script

preproc_scripts = []
problem_nums = []

for problem_folder in tqdm(code_file):
    scripts = os.listdir(os.path.join(code,problem_folder))
    problem_num = scripts[0].split('_')[0]
    for script in scripts:
        script_file = os.path.join(code,problem_folder,script)
        preprocessed_script = preprocess_script(script_file)

        preproc_scripts.append(preprocessed_script)
    problem_nums.extend([problem_num]*len(scripts))

100%|██████████| 300/300 [00:01<00:00, 183.54it/s]


코드 데이터에서 ''' ''' or """ """ 으로 긴 문장을 주석처리한 경우 삭제를 위해 추가로 전처리 진행

In [9]:
preproc_scripts
i = 0
for c in preproc_scripts:
      if "'''" in c: # code에 '''가 있으면 -> 다음 '''가 나타날 때 까지의 index로 slicing
        c = c[c.rfind("'''")+1:]
      if '"""' in c:
        c = c[c.rfind('"""')+1:]  # code에 """가 있으면 -> 다음 """가 나타날 때 까지의 index로 slicing
      preproc_scripts[i] = c
      i += 1

In [10]:
df = pd.DataFrame(data = {'code':preproc_scripts, 'problem_num':problem_nums})

In [11]:
tokenizer = AutoTokenizer.from_pretrained("microsoft/graphcodebert-base")
tokenizer.truncation_side = "left"
df['tokens'] = df['code'].apply(tokenizer.tokenize)
df['len'] = df['tokens'].apply(len)

Token indices sequence length is longer than the specified maximum sequence length for this model (773 > 512). Running this sequence through the model will result in indexing errors


In [12]:
ndf = df[df['len'] <= 512].reset_index(drop=True)
ndf.to_csv('train_data.csv')

In [13]:
ndf = pd.read_csv('train_data.csv',index_col = 0)

정제된 학습 데이터 저장하고 불러오기

In [14]:
train = []
valid = []

kfold = StratifiedKFold(random_state = 42, shuffle = True)

for idx, (train_idx, val_idx) in enumerate(kfold.split(ndf["code"], ndf['problem_num'])):
  X_train, X_val = ndf["code"].iloc[train_idx],  ndf["code"].iloc[val_idx]
  y_train, y_val = ndf['problem_num'].iloc[train_idx], ndf['problem_num'].iloc[val_idx]

  train_fold = pd.concat([X_train, y_train], axis = 1)
  valid_fold = pd.concat([X_val, y_val], axis = 1)
  train.append(train_fold)
  valid.append(valid_fold)

Fold 앙상블을 위해 정제된 데이터 나누기(실험 여건상 5 fold 진행)
이후의 코드는 fold 별로 나누어서 실행 5 fold라서 5번 반복

In [15]:
train_df = train[2].reset_index(drop=True)
valid_df = valid[2].reset_index(drop=True)

In [16]:
codes = train_df['code'].to_list()
problems = train_df['problem_num'].unique().tolist()
problems.sort()

tokenized_corpus = [tokenizer.tokenize(code) for code in codes]

In [17]:
train_df.problem_num.value_counts()

problem216    122
problem059    122
problem062    122
problem144    122
problem082    122
             ... 
problem210     72
problem209     67
problem112     64
problem113     63
problem034     57
Name: problem_num, Length: 300, dtype: int64

Pair data 생성 전, 각 problem의 개수 확인 57 ~ 122개 사이

In [18]:
total_positive_pairs= []
total_negative_pairs = []
for problem in tqdm(problems):
    solution_codes = train_df[train_df['problem_num'] == problem]['code']
    negative_codes = train_df[train_df['problem_num'] != problem]['code']
    positive_pairs = list(combinations(solution_codes.to_list(),2))
    positive_samples = random.sample(positive_pairs,round(len(positive_pairs)*0.025))

    negative_samples = []
    for i in range(round(len(positive_pairs)*0.025)):
      code1 = random.choice(solution_codes.to_list())
      code2 = random.choice(negative_codes.to_list())
      pair = (code1, code2)
      negative_samples.append(pair)
    
    total_positive_pairs.extend(positive_samples)
    total_negative_pairs.extend(negative_samples)

pos_code1 = list(map(lambda x:x[0],total_positive_pairs))
pos_code2 = list(map(lambda x:x[1],total_positive_pairs))

neg_code1 = list(map(lambda x:x[0],total_negative_pairs))
neg_code2 = list(map(lambda x:x[1],total_negative_pairs))

pos_label = [1]*len(pos_code1)
neg_label = [0]*len(neg_code1)

pos_code1.extend(neg_code1)
total_code1 = pos_code1
pos_code2.extend(neg_code2)
total_code2 = pos_code2
pos_label.extend(neg_label)
total_label = pos_label
pair_data = pd.DataFrame(data={
    'code1':total_code1,
    'code2':total_code2,
    'similar':total_label
})
pair_data = pair_data.sample(frac=1).reset_index(drop=True)

100%|██████████| 300/300 [00:23<00:00, 12.97it/s]


Pair data를 만드는 과정 -> Random Sampling
Positive pair - Problem number가 같은 것끼리 combination을 사용하여 pair를 만들고 그 중 특정 비율만큼 채택
ex) 같은 1번 문제의 코드가 100개 있을 경우 1번 문제의 combination 총 개수 (100x99)/2 = 4950개에서 특정 비율만큼 random하게 선택, 비율이 0.025면 4950x0.025 = 약124개 선택 -> 100개의 1번 문제 코드에서 124개의 정답지가 생성
Negative pair - Iteration에서 해당 번호의 코드에서 하나, 해당 번호가 아닌 코드에서 하나를 random하게 하나씩 채택
ex) 1번 문제의 iteration일 경우, positive pair에서 만들어진 124번만큼 1번 문제에서 하나를 선택하고, 2번~300번의 모든 데이터에서 random하게 하나의 코드 선택 => 생성된 정답지 만큼의 오답지 생성
비율 0.1 => 약 40만개 데이터 생성
비율 0.05 => 약 20만개 데이터 생성
비율 0.025 => 약 10만개 데이터 생성

In [19]:
pair_data.to_csv('train_data_fold4.csv', index = False)

In [20]:
codes = valid_df['code'].to_list()
problems = valid_df['problem_num'].unique().tolist()
problems.sort()

tokenized_corpus = [tokenizer.tokenize(code) for code in codes]

In [21]:
total_positive_pairs = []
total_negative_pairs = []

for problem in tqdm(problems):
    solution_codes = valid_df[valid_df['problem_num'] == problem]['code']
    negative_codes = valid_df[valid_df['problem_num'] != problem]['code']
    positive_pairs = list(combinations(solution_codes.to_list(),2))
    positive_samples = random.sample(positive_pairs,round(len(positive_pairs)*0.05))

    negative_samples = []
    for i in range(round(len(positive_pairs)*0.05)):
      code1 = random.choice(solution_codes.to_list())
      code2 = random.choice(negative_codes.to_list())
      pair = (code1, code2)
      negative_samples.append(pair)
    
    total_positive_pairs.extend(positive_samples)
    total_negative_pairs.extend(negative_samples)


pos_code1 = list(map(lambda x:x[0],total_positive_pairs))
pos_code2 = list(map(lambda x:x[1],total_positive_pairs))

neg_code1 = list(map(lambda x:x[0],total_negative_pairs))
neg_code2 = list(map(lambda x:x[1],total_negative_pairs))

pos_label = [1]*len(pos_code1)
neg_label = [0]*len(neg_code1)

pos_code1.extend(neg_code1)
total_code1 = pos_code1
pos_code2.extend(neg_code2)
total_code2 = pos_code2
pos_label.extend(neg_label)
total_label = pos_label
pair_data = pd.DataFrame(data={
    'code1':total_code1,
    'code2':total_code2,
    'similar':total_label
})
pair_data = pair_data.sample(frac=1).reset_index(drop=True)

100%|██████████| 300/300 [00:01<00:00, 218.69it/s]


Validation data의 생성 또한 같은 방법으로 진행

In [22]:
pair_data.to_csv('valid_data_fold4.csv', index = False)

Model - Microsoft/graphcodebert-base

In [23]:
MODEL = "microsoft/graphcodebert-base"

dataset = load_dataset('csv', data_files={'train': 'train_data_fold4.csv', 'test':  'valid_data_fold4.csv'})
MAX_LEN = 512

tokenizer = AutoTokenizer.from_pretrained(MODEL)
tokenizer.truncation_side = 'left'

Using custom data configuration default-c72add23350714be


  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

Dataset csv downloaded and prepared to /home/ubuntu/.cache/huggingface/datasets/csv/default-c72add23350714be/0.0.0/6b9057d9e23d9d8a2f05b985917a0da84d70c5dae3d22ddd8a3f22fb01c69d9e. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [24]:
def example_fn(examples):
    outputs = tokenizer(examples['code1'], examples['code2'], padding=True, max_length=MAX_LEN,truncation=True)
    if 'similar' in examples:
        outputs["labels"] = examples["similar"]
    return outputs

dataset = dataset.map(example_fn, remove_columns=['code1', 'code2', 'similar'])

_collator = DataCollatorWithPadding(tokenizer=tokenizer)
_metric = load_metric("glue", "sst2")

def metric_fn(p):
    preds, labels = p
    output =  _metric.compute(references=labels, predictions=np.argmax(preds, axis=-1))
    return output

Parameter 'function'=<function example_fn at 0x7f80f75c75e0> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


  0%|          | 0/102396 [00:00<?, ?ex/s]

  0%|          | 0/12578 [00:00<?, ?ex/s]

In [25]:
model = AutoModelForSequenceClassification.from_pretrained(MODEL)

args = TrainingArguments(
    output_dir ='runs/',
    per_device_train_batch_size=16,
    #gradient_accumulation_steps = 2,
    num_train_epochs=10,
    save_strategy="epoch",
    logging_strategy="epoch",
    evaluation_strategy="epoch",
    load_best_model_at_end= True,
    metric_for_best_model = 'accuracy',

)

trainer = Trainer(
        model=model,
        args=args,
        data_collator=_collator,
        train_dataset=dataset['train'],
        eval_dataset=dataset['test'],
        tokenizer=tokenizer,
        callbacks = [EarlyStoppingCallback(early_stopping_patience=3)],
        compute_metrics=metric_fn)

trainer.train()

Some weights of the model checkpoint at microsoft/graphcodebert-base were not used when initializing RobertaForSequenceClassification: ['lm_head.bias', 'lm_head.decoder.weight', 'lm_head.decoder.bias', 'lm_head.dense.bias', 'lm_head.layer_norm.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at microsoft/graphcodebert-base and are newly initialized: ['classifier.out_proj.bias', 'classifier.o

/home/ubuntu/anaconda3/lib/python3.9/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Epoch,Training Loss,Validation Loss,Accuracy
1,0.121800,0.065353,0.979488
2,0.048200,0.073771,0.982032
3,0.031200,0.060970,0.982589
4,0.019000,0.062232,0.985928
5,0.013000,0.071118,0.986405
6,0.008600,0.083101,0.987359
7,0.005900,0.106893,0.983066
8,0.003900,0.090328,0.987041
9,0.001600,0.091459,0.987677
10,0.000800,0.088187,0.988154


***** Running Evaluation *****
  Num examples = 12578
  Batch size = 24
Saving model checkpoint to runs/checkpoint-2134
Configuration saved in runs/checkpoint-2134/config.json
Model weights saved in runs/checkpoint-2134/pytorch_model.bin
tokenizer config file saved in runs/checkpoint-2134/tokenizer_config.json
Special tokens file saved in runs/checkpoint-2134/special_tokens_map.json
/home/ubuntu/anaconda3/lib/python3.9/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
***** Running Evaluation *****
  Num examples = 12578
  Batch size = 24
Saving model checkpoint to runs/checkpoint-4268
Configuration saved in runs/checkpoint-4268/config.json
Model weights saved in runs/checkpoint-4268/pytorch_model.bin
tokenizer config file saved in runs/checkpoint-4268/tokenizer_config.json
Special tokens fi

TrainOutput(global_step=21340, training_loss=0.025406165087234225, metrics={'train_runtime': 12343.8562, 'train_samples_per_second': 82.953, 'train_steps_per_second': 1.729, 'total_flos': 2.6932646409856058e+17, 'train_loss': 0.025406165087234225, 'epoch': 10.0})

Prediction

In [45]:
test_dataset = load_dataset("csv", data_files="/home/ubuntu/HwangJaewon/DL_class/test.csv")['train']
test_dataset = test_dataset.map(example_fn, remove_columns=['code1', 'code2'])

predictions = trainer.predict(test_dataset)

Using custom data configuration default-274b9386f5046d48
Reusing dataset csv (/home/ubuntu/.cache/huggingface/datasets/csv/default-274b9386f5046d48/0.0.0/6b9057d9e23d9d8a2f05b985917a0da84d70c5dae3d22ddd8a3f22fb01c69d9e)


  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/179700 [00:00<?, ?ex/s]

The following columns in the test set don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: pair_id. If pair_id are not expected by `RobertaForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 179700
  Batch size = 24


RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call,so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.

In [44]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

In [39]:
np.save('fold4_new.npy',list(predictions)[0])

NameError: name 'predictions' is not defined

In [ ]:
fold0 = np.load( 'fold0.npy')
fold1 = np.load('fold1.npy')
fold2 = np.load('fold2_new.npy')
fold3 = np.load('fold3.npy')
fold4 = np.load('fold4.npy')
fold0_200000 = np.load('pred1.npy')
fold0_400000 = np.load( 'pred_fold0.npy')
pred = (fold0 + fold1+ fold2 + fold3 + fold4 + fold0_200000 + fold0_400000)/7

저장한 prediction ensemble
모두 합쳤을 때 가장 좋은 결과

In [ ]:
df = pd.read_csv("sample_submission.csv")
df['similar'] = np.argmax(pred, axis=-1)
df.to_csv('ensem_all.csv', index=False)



In [ ]:
df['similar'].value_counts()